In [4]:
from src.matrikadb import connect_to_db

db = connect_to_db()

Successfully connected to the database!


In [5]:

def get_table_schema(cursor, table_name):
    # Get column information
    columns = cursor.columns(table=table_name).fetchall()
    create_table = f"CREATE TABLE {table_name} (\n"
    
    for i, column in enumerate(columns):
        col_name = column[3]  # COLUMN_NAME is at index 3
        col_type = column[5]  # TYPE_NAME is at index 5
        col_size = column[6]  # COLUMN_SIZE is at index 6
        nullable = "NULL" if column[10] else "NOT NULL"  # NULLABLE is at index 10
        
        # Handle specific data types
        if col_type.lower() in ('varchar', 'nvarchar', 'char', 'nchar'):
            col_type = f"{col_type}({col_size})"
        elif col_type.lower() == 'decimal':
            precision = column[6]    # COLUMN_SIZE
            scale = column[8]        # DECIMAL_DIGITS is at index 8
            col_type = f"{col_type}({precision},{scale})"
            
        create_table += f"    {col_name} {col_type} {nullable}"
        if i < len(columns) - 1:
            create_table += ","
        create_table += "\n"
    
    create_table += ");\n"
    return create_table

# ... rest of the code remains the same ...

def dump_schema(conn):
    cursor = conn.cursor()
    schema_sql = []
    
    # Get all tables
    tables = cursor.tables(tableType='TABLE').fetchall()
    
    for table in tables:
        if table.table_type == 'TABLE':  # Skip system tables
            table_name = table.table_name
            schema_sql.append(f"\n-- Table: {table_name}")
            schema_sql.append(get_table_schema(cursor, table_name))
            
            # Get indexes
            cursor.execute(f"""
                SELECT 
                    i.name AS index_name,
                    c.name AS column_name,
                    i.is_unique,
                    i.is_primary_key
                FROM sys.indexes i
                INNER JOIN sys.index_columns ic ON i.object_id = ic.object_id AND i.index_id = ic.index_id
                INNER JOIN sys.columns c ON ic.object_id = c.object_id AND ic.column_id = c.column_id
                WHERE OBJECT_NAME(i.object_id) = '{table_name}'
                AND i.name IS NOT NULL
            """)
            
            for idx in cursor.fetchall():
                if idx.is_primary_key:
                    schema_sql.append(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({idx.column_name});")
                elif idx.is_unique:
                    schema_sql.append(f"CREATE UNIQUE INDEX {idx.index_name} ON {table_name}({idx.column_name});")
                else:
                    schema_sql.append(f"CREATE INDEX {idx.index_name} ON {table_name}({idx.column_name});")
    
    # Get views
    cursor.execute("""
        SELECT name, definition 
        FROM sys.objects o 
        JOIN sys.sql_modules m ON o.object_id = m.object_id
        WHERE type = 'V'
    """)
    
    for view in cursor.fetchall():
        schema_sql.append(f"\n-- View: {view.name}")
        schema_sql.append(f"{view.definition}\n")
    
    # Write to file
    with open('schema_dump.sql', 'w', encoding='utf-8') as f:
        f.write('\n'.join(schema_sql))

# Use your existing connection
try:
    dump_schema(db)
    print("Schema successfully dumped to schema_dump.sql")
except Exception as e:
    print(f"Error dumping schema: {e}")

Schema successfully dumped to schema_dump.sql
